In [ ]:
# Auto-reload frequently changed files
%load_ext autoreload
%autoreload 2
%aimport utils

import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact
from os.path import join

from constants import COLUMNS
from utils import (
    read_combined_daily_counts_df,
    read_combined_demographics_df, 
    read_combined_diagnoses_df,
    read_combined_labs_df,
    apply_theme
)

# Daily Counts

In [ ]:
# TODO: Better way to clear the codes here?
df = read_combined_daily_counts_df()

# Add range value (under, mid, upper):
# (1) add upper value (i.e., X + Upper_Bound_X)
df["upper_" + COLUMNS.NEW_POSITIVE_CASES] = df[COLUMNS.NEW_POSITIVE_CASES] + df[COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES]
df["upper_" + COLUMNS.PATIENTS_IN_ICU] = df[COLUMNS.PATIENTS_IN_ICU] + df[COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU]
df["upper_" + COLUMNS.NEW_DEATHS] = df[COLUMNS.NEW_DEATHS] + df[COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS]

# (2) add under value (i.e., X)
df["under_" + COLUMNS.NEW_POSITIVE_CASES] = df[COLUMNS.NEW_POSITIVE_CASES]
df["under_" + COLUMNS.PATIENTS_IN_ICU] = df[COLUMNS.PATIENTS_IN_ICU]
df["under_" + COLUMNS.NEW_DEATHS] = df[COLUMNS.NEW_DEATHS]

# (3) modify mid value (i.e., X + Upper_Bound_X / 2)
df[COLUMNS.NEW_POSITIVE_CASES] = df[COLUMNS.NEW_POSITIVE_CASES] + df[COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES] / 2.0
df[COLUMNS.PATIENTS_IN_ICU] = df[COLUMNS.PATIENTS_IN_ICU] + df[COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU] / 2.0
df[COLUMNS.NEW_DEATHS] = df[COLUMNS.NEW_DEATHS] + df[COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS] / 2.0

# Drop unused columns
df = df.drop(columns=[
    COLUMNS.UNMASKED_SITES_NEW_POSITIVE_CASES,
    COLUMNS.UNMASKED_SITES_PATIENTS_IN_ICU,
    COLUMNS.UNMASKED_SITES_NEW_DEATHS,
    COLUMNS.MASKED_SITES_NEW_POSITIVE_CASES,
    COLUMNS.MASKED_SITES_PATIENTS_IN_ICU,
    COLUMNS.MASKED_SITES_NEW_DEATHS,
    COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES,
    COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU,
    COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS
])

# Wide to long
CATEGORY = "category"
df = pd.melt(df, id_vars=[
    COLUMNS.SITE_ID, COLUMNS.DATE, 
    "upper_" + COLUMNS.NEW_POSITIVE_CASES, "upper_" + COLUMNS.PATIENTS_IN_ICU, "upper_" + COLUMNS.NEW_DEATHS,
    "under_" + COLUMNS.NEW_POSITIVE_CASES, "under_" + COLUMNS.PATIENTS_IN_ICU, "under_" + COLUMNS.NEW_DEATHS
])
df = df.rename(columns={"variable": CATEGORY, "value": COLUMNS.NUM_PATIENTS})

# Leave only the 'upper' and 'under' values for the certain 'category' only
df["upper"] = -1
df["under"] = -1

df.loc[df[CATEGORY] == COLUMNS.NEW_POSITIVE_CASES, "upper"] = df.loc[df[CATEGORY] == COLUMNS.NEW_POSITIVE_CASES, "upper_" + COLUMNS.NEW_POSITIVE_CASES]
df.loc[df[CATEGORY] == COLUMNS.PATIENTS_IN_ICU, "upper"] = df.loc[df[CATEGORY] == COLUMNS.PATIENTS_IN_ICU, "upper_" + COLUMNS.PATIENTS_IN_ICU]
df.loc[df[CATEGORY] == COLUMNS.NEW_DEATHS, "upper"] = df.loc[df[CATEGORY] == COLUMNS.NEW_DEATHS, "upper_" + COLUMNS.NEW_DEATHS]
df.loc[df[CATEGORY] == COLUMNS.NEW_POSITIVE_CASES, "under"] = df.loc[df[CATEGORY] == COLUMNS.NEW_POSITIVE_CASES, "under_" + COLUMNS.NEW_POSITIVE_CASES]
df.loc[df[CATEGORY] == COLUMNS.PATIENTS_IN_ICU, "under"] = df.loc[df[CATEGORY] == COLUMNS.PATIENTS_IN_ICU, "under_" + COLUMNS.PATIENTS_IN_ICU]
df.loc[df[CATEGORY] == COLUMNS.NEW_DEATHS, "under"] = df.loc[df[CATEGORY] == COLUMNS.NEW_DEATHS, "under_" + COLUMNS.NEW_DEATHS]

# Drop unused columns for the last time
df = df.drop(columns=[
    "upper_" + COLUMNS.NEW_POSITIVE_CASES, "upper_" + COLUMNS.PATIENTS_IN_ICU, "upper_" + COLUMNS.NEW_DEATHS,
    "under_" + COLUMNS.NEW_POSITIVE_CASES, "under_" + COLUMNS.PATIENTS_IN_ICU, "under_" + COLUMNS.NEW_DEATHS
])

df.head()

In [ ]:
MID_FIELDS = [COLUMNS.NEW_POSITIVE_CASES, COLUMNS.NEW_DEATHS, COLUMNS.PATIENTS_IN_ICU]
UPPER_FIELDS = [f"upper_{COLUMNS.NEW_POSITIVE_CASES}", f"upper_{COLUMNS.NEW_DEATHS}", f"upper_{COLUMNS.PATIENTS_IN_ICU}"]
UNDER_FIELDS = [f"under_{COLUMNS.NEW_POSITIVE_CASES}", f"under_{COLUMNS.NEW_DEATHS}", f"under_{COLUMNS.PATIENTS_IN_ICU}"]
THREE_COLORS = ["#CA2026", "#377FB8", "#60B75D"]
COLOR_BY_FIELD = {
    COLUMNS.NEW_POSITIVE_CASES: "#CA2026",
    COLUMNS.NEW_DEATHS: "#60B75D",
    COLUMNS.PATIENTS_IN_ICU: "#377FB8"
}
GRAY_COLOR = "lightgray"

line = alt.Chart(df).mark_line(size=3).encode(
    x=alt.X(
        f"{COLUMNS.DATE}:T", axis=alt.Axis(tickCount=7), 
        title=None,
    ),
    y=alt.Y(
        f"{COLUMNS.NUM_PATIENTS}:Q", axis=alt.Axis(tickCount=5), 
        title="Number of patients",
        scale=alt.Scale(domain=[-1,170])
    ),
    color=alt.Color(f"{CATEGORY}:N", scale=alt.Scale(domain=MID_FIELDS, range=THREE_COLORS), legend=alt.Legend(title=None))
)
point = line.mark_circle(size=40)
errorband = alt.Chart(df).mark_errorband().encode(
    x=alt.X(
        f"{COLUMNS.DATE}:T", axis=alt.Axis(tickCount=7), 
        title=None,
    ),y=alt.Y(
        f"upper:Q", title=""
    ),y2=alt.Y2(
        f"under:Q"
    ),
    color=alt.Color(f"{CATEGORY}:N", scale=alt.Scale(domain=MID_FIELDS, range=THREE_COLORS))
)

agg_chart = (line + point + errorband).resolve_scale(color="shared")

apply_theme(agg_chart).properties(
    width=750, height=400, title="Number of Positive Cases, Patients in ICU, and Deaths"
).interactive()

# Demographics

In [ ]:
# TODO: Better way to clear the codes here?
df = read_combined_demographics_df()

print(df)